In [1]:
# Load BeautifulSoup, and pandas to store in a dataframe
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep
from tqdm import tqdm

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By




In [77]:
driver = webdriver.Chrome()
driver.get("https://www.scrapethissite.com/pages/ajax-javascript/")

In [98]:
button_2015 = driver.find_element(by=By.ID, value="2015")
button_2014 = driver.find_element(by=By.ID, value="2014")
button_2015.click()
sleep(3)

In [96]:
a = driver.find_element(by=By.TAG_NAME, value="table")
a.find_element(by=By.CLASS_NAME, value="film").get_attribute('innerHTML')

'<td class="film-title">Spotlight  </td><td class="film-nominations">6</td><td class="film-awards">2</td><td class="film-best-picture"><i class="glyphicon glyphicon-flag"></i></td>'

In [92]:
a

<selenium.webdriver.remote.webelement.WebElement (session="8410ce34edc4c69d93e454c1f86c2139", element="f.14D2F0722D433E43FB8D377573382587.d.A80F0E4B82E48F004597995AE1FE7707.e.48")>

In [19]:
button_2015.get_attribute("class")

'year-link'

In [16]:
help(By)

Help on class By in module selenium.webdriver.common.by:

class By(builtins.object)
 |  Set of supported locator strategies.
 |
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  CLASS_NAME = 'class name'
 |
 |  CSS_SELECTOR = 'css selector'
 |
 |  ID = 'id'
 |
 |  LINK_TEXT = 'link text'
 |
 |  NAME = 'name'
 |
 |  PARTIAL_LINK_TEXT = 'partial link text'
 |
 |  TAG_NAME = 'tag name'
 |
 |  XPATH = 'xpath'



In [101]:
html_2015 = driver.page_source
driver.quit()

KeyboardInterrupt: 

In [102]:
soup = BeautifulSoup(html_2015, 'html.parser')

In [31]:
print(soup.prettify())

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Oscar Winning Films: AJAX and Javascript | Scrape This Site | A public sandbox for learning web scraping
  </title>
  <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Click through a bunch of great films. Learn how content is added to the page asynchronously with Javascript and how you can scrape it." name="description"/>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
  <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
  <meta content="noindex" name=

In [104]:
print(soup.find(class_='film-title'))

<td class="film-title">Spotlight  </td>


In [106]:
print(soup.find(class_='film').prettify())

<tr class="film">
 <td class="film-title">
  Spotlight
 </td>
 <td class="film-nominations">
  6
 </td>
 <td class="film-awards">
  2
 </td>
 <td class="film-best-picture">
  <i class="glyphicon glyphicon-flag">
  </i>
 </td>
</tr>



## Scraping the entire table

In [54]:
from io import StringIO
pd.read_html(StringIO(html))

[                                            Title  Nominations  Awards  \
 0                                       Spotlight            6       2   
 1                              Mad Max: Fury Road           10       6   
 2                                    The Revenant           12       3   
 3                                 Bridge of Spies            6       1   
 4                                   The Big Short            5       1   
 5                                 The Danish Girl            4       1   
 6                                            Room            4       1   
 7                                      Ex Machina            2       1   
 8                               The Hateful Eight            2       1   
 9                                      Inside Out            2       1   
 10                                            Amy            1       1   
 11                                     Bear Story            1       1   
 12  A Girl in the River:

In [64]:
# Using the same methods we've used until now to extract table data
# This data should be easier to extract as each `<td>` has a unique 'class' attribute that will let us extract the entire column at once using the `.find_all()` method, instead of looping over rows
# Get the elements, but form this we will need to extract the text
titles_result = soup.find_all(class_="film-title")
nominations_result = soup.find_all(class_="film-nominations")
awards_result = soup.find_all(class_="film-awards")
best_picture_result = soup.find_all(class_="film-best-picture")

# Check that the same number of rows where extracted for all columns
print("Number of elements found for each class", len(titles_result), len(titles_result), len(titles_result), len(titles_result))


# They all have 16 rows, so we can continue and make a table with this data
list_movies = []
for item in range(len(titles_result)):
    dict_movies = {}
    dict_movies["titles"] = titles_result[item].get_text()
    dict_movies["nominations"] = nominations_result[item].get_text()
    dict_movies["awards"] = awards_result[item].get_text()
    if best_picture_result[item].find("i") == None:
        bp_value = "No"
    else:
        bp_value = "Yes"
    dict_movies['best_picture'] = bp_value
    list_movies.append(dict_movies)

movies_2015 = pd.DataFrame(list_movies)

Number of elements found for each class 16 16 16 16


17 lines of code

In [65]:
movies_2015

,titles,nominations,awards,best_picture
0,Spotlight,6,2,Yes
1,Mad Max: Fury Road,10,6,No
2,The Revenant,12,3,No
3,Bridge of Spies,6,1,No
4,The Big Short,5,1,No
5,The Danish Girl,4,1,No
6,Room,4,1,No
7,Ex Machina,2,1,No
8,The Hateful Eight,2,1,No
9,Inside Out,2,1,No


In [ ]:
titles_lc = [elem.get_text() for elem in soup.find_all(class_="film-title")]
nominations_lc = [elem.get_text() for elem in soup.find_all(class_="film-nominations")]
awards_lc = [elem.get_text() for elem in soup.find_all(class_="film-awards")]
best_picture_lc = ["Yes" if elem.find("i") == None else "No" for elem in soup.find_all(class_="film-best-picture")]
movies_2015_v2 = pd.DataFrame(
    {'titles': titles_lc, 'nominations': nominations_lc, 'awards': awards_lc, 'best_picture': best_picture_lc}
)

5 lines of code

In [75]:
movies_2015_v2

,titles,nominations,awards,best_picture
0,Spotlight,6,2,No
1,Mad Max: Fury Road,10,6,Yes
2,The Revenant,12,3,Yes
3,Bridge of Spies,6,1,Yes
4,The Big Short,5,1,Yes
5,The Danish Girl,4,1,Yes
6,Room,4,1,Yes
7,Ex Machina,2,1,Yes
8,The Hateful Eight,2,1,Yes
9,Inside Out,2,1,Yes


## Challenges
- Challenge 1: Now modify your code to add a loop that scrapes all tables for years 2010-2015 in just one dataframe.
- Challenge 2: From this page "https://www.scrapingcourse.com/javascript-rendering", extract all product names, price and hyperlinks to the detailed view of the product. When you complete that, and if you are up to an additional challenge, scrape from the detailed view of the product the SKU, the Category and the Description.

In [9]:
text_box = driver.find_element(by=By.NAME, value="my-text")
submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

In [11]:
text_box.send_keys("Selenium")
submit_button.click()

In [12]:
message = driver.find_element(by=By.ID, value="message")
text = message.text

driver.quit()

In [13]:
text

'Received!'